<img src="images/logos/TClogo.png">

### in colaboration with

<img src="images/robotnik.png">

### presents

# ROS Developers Live Class n51

# How to fuse odometry & IMU with *robot_localization*

This notebook is additional material for the **ROS Developers Live Class n.51** created and provided for free by **Alberto Ezquerro** and **Ricardo Tellez** of <a href="www.theconstructsim.com">The Construct</a>. You can distribute this notebook as long as you provide copy of this paragraph with it.

## Why this class?

In today's live class, we are going to learn the following contents:
1. Why it is needed to fuse sensor data for navigation
2. What is the *robot_localization* package
3. How to use the *robot_localization* package for sensor fusion

Pre-requisites for this live class are:
* Basic knowledge of ROS concepts such as topics, publish and subscribe, ROS Service
* Know how to create a map and how to localize a robot in it. <a href="https://youtu.be/RknTTpga64s">Check Live Class n49 if you don't know how</a>
* Love for Robotics 
* ...that's it!!

## Summit XL robot by Robotnik

<img src="images/Summitxl_sim2.png">

For this class we are going to use the Summit XL robot by <a href="">Robotnik</a>. That is a robot that has the following characteristics:

* Four wheels
* Laser ranger
* IMU
* Camera
* GPS

In case you are interested on this robot, you can <a href="https://www.robotnik.eu/mobile-robots/summit-xl/">find more information here</a>.

<img src="images/Summitxl_real.jpg">

### How to use this ROSject

A <a href="http://rosjects.com">**ROSject**</a> is a **ROS project** packaged in such a way that all the material it contains (**ROS code, Gazebo simulations and Notebooks**) can be shared with any body **using only a web link**. That is what we did with all the attendants to the Live Class, we shared this ROSject with them (so they can have access to all the ROS material they contain).

**Check <a heref="https://www.youtube.com/watch?v=cR-Ow5K7oSo">this webinar</a> to learn more about ROSjects and how to create your own ROSjects**.

You will need to have a free account at the <a href="http://rosds.online">ROS Development Studio</a> (ROSDS). Get the account and then follow the indications below.

## 1. Why do we need to fuse sensor data for autonomous robot navigation?

Because sensor data is noisy, which produces an error in the localization of the robot.

When talking about odometry, the better the odometry, the better the localization of the robot.

The problem with errors in odometry is that they accummulate over time.

<img src="images/comparison.png">

## 2. What is the *robot_localization* package

It is a package for mixing different sources of odometry into a more stable one.

<img src="images/robot_localization_graph1.png" width="600" />

## 3. Let's launch Summit XL simulation

For that you need to go to **Simulations->Summit XL**. After that you should get the following simulation

<img src="images/Summitxl_sim1.png">

## 4. Let's launch the code that adds noise to odometry

Check in Rviz how the odometry looks:

#### Launch the Rviz to localize the robot in the map

In [ ]:
$ rosrun rviz rviz

Load the config file for Rviz that is provided in this rosject at */home/user/odom_config.rviz*

#### Launch the noisy_odom package

In [ ]:
$ rosrun noisy_odom add_noise.py

<img src="images/odom1.png">

## 5. Let's configure the *robot_localization* package

Let's create a new ROS package that we are going to call **summit_odometry**.

In [ ]:
$ cd ~/catkin_ws/src
$ catkin_create_pkg summit_odometry
$ cd summit_odometry
$ mkdir launch
$ mkdir config

Then create a launch file named *start_filter.launch* with the following content:

In [ ]:
<launch> 

    <!-- Run the EKF Localization node -->
    <node pkg="robot_localization" type="ekf_localization_node" name="ekf_localization">
        <rosparam command="load" file="$(find summit_odometry)/config/ekf_localization.yaml"/>
    </node>

</launch>

Basically, what we are doing here is to launch the ROS program named **ekf_localization_node** (which uses the Extended Kalman Filter), from the **robot_localization** package with a specific configuration file named **ekf_localization.yaml**.

### The configuration file

Now, inside this folder, create the configuration file, named **ekf_localization.yaml**. Inside this file, you will place the following configuration. Complete all the parameters that are with an interrogation sign (?).

In [ ]:
#Configuation for robot odometry EKF
#
frequency: 50
    
two_d_mode: true
    
publish_tf: false

# Complete the frames section 
odom_frame: ?
base_link_frame: ?
world_frame: ?
map_frame: ?

# Complete the odom0 configuration
odom0: ?
odom0_config: [?, ?, ?,
               ?, ?, ?,
               ?, ?, ?,
               ?, ?, ?,
               ?, ?, ?,]
odom0_differential: false

# Complete the imu0 configuration
imu0: ?
imu0_config: [?, ?, ?,
              ?, ?, ?,
              ?, ?, ?,
              ?, ?, ?,
              ?, ?, ?,]
imu0_differential: false

process_noise_covariance": [0.05, 0,    0,    0,    0,    0,    0,     0,     0,    0,    0,    0,    0,    0,    0,
                                              0,    0.05, 0,    0,    0,    0,    0,     0,     0,    0,    0,    0,    0,    0,    0,
                                              0,    0,    0.06, 0,    0,    0,    0,     0,     0,    0,    0,    0,    0,    0,    0,
                                              0,    0,    0,    0.03, 0,    0,    0,     0,     0,    0,    0,    0,    0,    0,    0,
                                              0,    0,    0,    0,    0.03, 0,    0,     0,     0,    0,    0,    0,    0,    0,    0,
                                              0,    0,    0,    0,    0,    0.06, 0,     0,     0,    0,    0,    0,    0,    0,    0,
                                              0,    0,    0,    0,    0,    0,    0.025, 0,     0,    0,    0,    0,    0,    0,    0,
                                              0,    0,    0,    0,    0,    0,    0,     0.025, 0,    0,    0,    0,    0,    0,    0,
                                              0,    0,    0,    0,    0,    0,    0,     0,     0.04, 0,    0,    0,    0,    0,    0,
                                              0,    0,    0,    0,    0,    0,    0,     0,     0,    0.01, 0,    0,    0,    0,    0,
                                              0,    0,    0,    0,    0,    0,    0,     0,     0,    0,    0.01, 0,    0,    0,    0,
                                              0,    0,    0,    0,    0,    0,    0,     0,     0,    0,    0,    0.02, 0,    0,    0,
                                              0,    0,    0,    0,    0,    0,    0,     0,     0,    0,    0,    0,    0.01, 0,    0,
                                              0,    0,    0,    0,    0,    0,    0,     0,     0,    0,    0,    0,    0,    0.01, 0,
                                              0,    0,    0,    0,    0,    0,    0,     0,     0,    0,    0,    0,    0,    0,    0.015]


initial_estimate_covariance: [1e-9, 0,    0,    0,    0,    0,    0,    0,    0,    0,     0,     0,     0,    0,    0,
                                                      0,    1e-9, 0,    0,    0,    0,    0,    0,    0,    0,     0,     0,     0,    0,    0,
                                                      0,    0,    1e-9, 0,    0,    0,    0,    0,    0,    0,     0,     0,     0,    0,    0,
                                                      0,    0,    0,    1e-9, 0,    0,    0,    0,    0,    0,     0,     0,     0,    0,    0,
                                                      0,    0,    0,    0,    1e-9, 0,    0,    0,    0,    0,     0,     0,     0,    0,    0,
                                                      0,    0,    0,    0,    0,    1e-9, 0,    0,    0,    0,     0,     0,     0,    0,    0,
                                                      0,    0,    0,    0,    0,    0,    1e-9, 0,    0,    0,     0,     0,     0,    0,    0,
                                                      0,    0,    0,    0,    0,    0,    0,    1e-9, 0,    0,     0,     0,     0,    0,    0,
                                                      0,    0,    0,    0,    0,    0,    0,    0,    1e-9, 0,     0,     0,     0,    0,    0,
                                                      0,    0,    0,    0,    0,    0,    0,    0,    0,    1e-9,  0,     0,     0,    0,    0,
                                                      0,    0,    0,    0,    0,    0,    0,    0,    0,    0,     1e-9,  0,     0,    0,    0,
                                                      0,    0,    0,    0,    0,    0,    0,    0,    0,    0,     0,     1e-9,  0,    0,    0,
                                                      0,    0,    0,    0,    0,    0,    0,    0,    0,    0,     0,     0,     1e-9, 0,    0,
                                                      0,    0,    0,    0,    0,    0,    0,    0,    0,    0,     0,     0,     0,    1e-9, 0,
                                                      0,    0,    0,    0,    0,    0,    0,    0,    0,    0,     0,     0,     0,    0,    1e-9]




### The reference frames

* **base_link_frame**: It is the frame that is in the robot itself, to which any sensor can be referenced. It is usually located in the center of the robot. It travels with it.


* **odom_frame**: It is the frame that is used to report the odometry.


* **map_frame**: It is the frame that is used to report a global position from a system that knows where the robot is. For instance an AMCL system. If you are not using any external Localization system, the this can be ignored.


* **world_frame**: It is the frame that references which one of the two previous frames is going to be used to get the absolute coordinates of the robot in the world.

In our case that would be:

In [ ]:
base_link_frame: base_link
odom_frame: odom
world_frame: odom

### Adding sensors to fuse

Any sensor that produces messages in any of these formats, can be fed to the robot_localization package to estimate the robot position.

* <a href="http://docs.ros.org/melodic/api/nav_msgs/html/msg/Odometry.html" target="_blank">nav_msgs/Odometry</a>

* <a href="http://docs.ros.org/melodic/api/nav_msgs/html/msg/Odometry.html" target="_blank">sensor_msgs/Imu</a>

* <a href="http://docs.ros.org/lunar/api/geometry_msgs/html/msg/PoseWithCovarianceStamped.html" target="_blank">geometry_msgs/PoseWithCovarianceStamped</a>

* <a href="http://docs.ros.org/melodic/api/geometry_msgs/html/msg/TwistWithCovarianceStamped.html" target="_blank">geometry_msgs/TwistWithCovarianceStamped</a>

And, most importantly, you can have many of each sensor. This means that, for instance, you can have the odometry provided by two different sensors (wheel encoders and visual odometry), or two different Inertial Measurement Units.

#### a. First indicate the topic of the sensor

In our case are:

* /noisy_odom
* /imu/data

#### b. Configure the variables matrix

Now, from each of the input sensors, we must specify which components of their messages are going to be merged (fused) in the Kalman Filter to compute the final state estimation. To specify this, you must fill a 3x5 value matrix. The matrix means the following:

In [ ]:
[  X,        Y,       Z
  roll,    pitch,    yaw
  X/dt,     Y/dt,    Z/dt
 roll/dt, pitch/dt, yaw/dt
  X/dt2,    Y/dt2,   Z/dt2]

The above values mean the following:

* **X, Y, Z**: These are the [x,y,z] coordinates of the robot.
* **roll, pitch, yaw**: These are the rpy axis, which specify the orientation of the robot.
* **X/dt, Y/dt, Z/dt**: These are the velocities of the robot.
* **roll/dt, pitch/dt, yaw/dt**: These are the angular velocities of the robot
* **X/$dt^2$, Y/$dt^2$, Z/$dt^2$**: These are the linear accelerations of the robot:

So, in this case, the values we are taking into account for the Kalman Filter are, for the odometry data:

* linear velocities in X and Y, and angular velocity in Z.

In [ ]:
odom0_config: [false, false, false
 false, false, false
 true,  true,  false
 false, false, true
 false, false, false,]

And for the IMU data:

* yaw(orientation), angular velocity in Z and linear acceleration in X. 

#### c. Covariance matrices

Use the default values

* **process_noise_covariance**: This parameter is used to model uncertainty in the prediction stage of the filtering algorithms. WHAT!!?? Basically, it's used to improve the results produced by the filter. The values on the diagonals are the variances for the state vector, which include pose, then velocities, then linear acceleration. It is not mandatory to set, but you will achieve superior results by tunning it. Anyways, unless you are an expert on the matter, it's not easy to set at all. So, the best option in that case would be to test different values, and see how they improve or decrease the results.


* **initial_state_covariance**: The estimate covariance defines the error in the current state estimate. This parameter allows to set the initial value for the matrix, which will affect how quickly the filter converges.
Here's the rule you should follow: if you are measuring a variable, make the diagonal value in initial_estimate_covariance larger than that measurement's covariance. So, for example, if your measurement's covariace value for the variable in question is 1e-6, make the initial_estimate_covariance diagonal value 1e-3 or something like that.

## 6. Launch the *robot_localization*

Now it is time to launch the *robot_localization* with your configuration:

In [ ]:
$ roslaunch summit_odometry start_filter.launch

Check now in Rviz that the odometry is now filtered and stabilized. You can find the result in topic:

*/odom/filtered*

## 7. How to connect to the real robot

#### IMPORTANT: kill the simulation window to prevent interferences

### 1. Install Husarnet communication package in the robot

First you need to install a package in the robot itself which allows IP6 communication with ROSDS. This is possible thanks to Husarnet.

The package has been already installed in the robot by our team.

### 2. Start the communication script in the robot

You need to start the *set_etc_hosts_mode.py* script to put the robot in communication mode.

This has been already done by our team.

In [ ]:
$ sudo python set_etc_hosts_mode.py RemoteMode summit

At this point in time, you should ask the robot for its communication address by issuing the following command:

In [ ]:
$ sudo husarnet websetup

Also get the hostname of the computer of the robot:

In [ ]:
$ hostname

### 3. Start the robot network in ROSDS

Go to the top menu and select **Real Robot->Robot Network**.

After a few seconds you should see that you have started the robot network of ROSDS.

<img src="images/robot_network.png">

### 4. Add a connection to a robot in ROSDS

Now add a new connection to a robot by using the top menu and select **Real Robot->Add device**.

On the dialog that will appear, indicate the **hostname** of the robot's computer, and the **communication web address** that you got on step 1.

After a few seconds you should see the following configuration.

<img src="images/robot_network.png">

### 5. Set the master to the summit

Now you have to indicate to ROSDS that the ROS_MASTER_URI should be pointing to the robot and not to the ROSDS itself.

For that, 

* first, we recommend to kill the simulation to prevent conflic with the topics of the real robot. 

* Then, go to **Real Robot** and click on the circle of the **summit**.

**From this point, the ROSDS is connected to the real robot!!!**

You can check that the topics in a shell correspond to the ones in the robot.

In [ ]:
$ rostopic list

### 6. Re-launch the *robot_localization*

In [ ]:
$ roslaunch summit_odometry start_filter.launch

### 7. Check in rviz the results

In [ ]:
$ rosrun rviz rviz

* Mission completed!!

### Before you log off, remember to <span style="background: #098be8; padding: 10px; color:white;">GIVE US A LIKE</span> and hit the <span style="background: #098be8; padding: 10px; color:white;">THUMBS UP</span> and <span style="background: #098be8; padding: 10px; color:white;">SUBSCRIBE</span> for more weekly tutorials!!!

We have an online academy that teaches you more about how to make robots navigate with ROS using GPS or laser sensors, as well as all the details of all the parameters of the configuration. Check the following related courses:
* <a href="http://www.theconstructsim.com/construct-learn-develop-robots-using-ros/robotigniteacademy_learnros/ros-courses-library/ros-robot-localization-package/">Fuse sensor data to improve localization</a>
* <a href="https://goo.gl/iog3x9">Mastering with ROS: Summit XL robot</a>


 <a href="https://goo.gl/7ApVAp"><img src="images/logos/RIAlogo.png"></a>

<img src="images/rosdevcon2.png">

A **hands on conference** where you will **learn and practice** at the same time **with the speakers**. 

It is an online conference with the same format of the Live Classes. You can attend from anywhere and will get a **rosject** with all the content of the speakers.

#### 8 speakers - 8 practical ROS projects on a single weekend

### You can also be a speaker of the conference. Check the call for papers

### More information here: <a href="http://www.rosdevcon.com">www.rosdevcon.com</a>

### Check the videos of the previous ROSDevCon 2018 <a href="">here</a>

# KEEP PUSHING YOUR ROS LEARNING!